In [7]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import transformers
import torch
# bertscore = load("bertscore") 
tokenizer =  AutoTokenizer.from_pretrained('/ssd_scratch/cvit/aparna/control_bart/checkpoint-6000',cache_dir='/ssd_scratch/cvit/aparna/bart')
#model = AutoModelForSeq2SeqLM.from_pretrained('/ssd_scratch/cvit/aparna/results/checkpoint-6000',cache_dir='/ssd_scratch/cvit/aparna/bart').to('cuda:1')
model = AutoModelForSeq2SeqLM.from_pretrained('/ssd_scratch/cvit/aparna/control_bart/checkpoint-6000',cache_dir='/ssd_scratch/cvit/aparna/bart').to('cuda:1')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
transformers.__version__

'4.35.2'

In [8]:
import evaluate
import numpy as np
from tqdm import tqdm
import json
from nltk.tokenize import RegexpTokenizer
bleu_metric = evaluate.load("sacrebleu")
sari_metric = evaluate.load("sari")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    print(len(eval_arg[0]),len(eval_arg[1]),len(eval_arg[2]))
    print(len(eval_arg))
    text_inputs = eval_arg[0]
    text_preds = eval_arg[1]
    text_labels = eval_arg[2]
    texts_bleu =[text.strip() for text in text_preds]
    labels_bleu = [[text.strip()] for text in text_labels[0]]
    result = bleu_metric.compute(predictions=texts_bleu, references=text_inputs)
    return result["score"],sari_metric.compute(
        predictions=text_preds,
        references=text_labels,
        sources=text_inputs,
    )['sari']

In [21]:
the_encoder = model.get_encoder()
the_decoder = model.get_decoder()

last_linear_layer = model.lm_head



In [9]:
import pandas as pd
test_data = pd.read_csv("../../data/10/test.csv")
test_data = test_data.dropna()
test_data = test_data.reset_index(drop=True)
#take 1000 samples
test_data = test_data[:10000]
texts = test_data["source"].tolist()
labels = test_data["target"].tolist()


metrics =[]
inpu = []
cands = []
lab = []
max_l = 512
num_b = 10
num_sub_b =1

for  i  in tqdm(range(len(texts))):
    text = texts[i]
    # encode the text into tensor of integers using the appropriate tokenizer
    inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True)

    # generate text until the output length (which includes the context length) reaches 50
    beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
        no_repeat_ngram_size=3,
        num_return_sequences=num_b,
        top_k=4, top_p=0.95
        # return_dict_in_generate=True,
    )
   # print("====================================")
    inpu.extend([text]*10)
    c =[]
    for x, beam in enumerate(beam_outputs):
        #print("{}".format(i, tokenizer.decode(beam, skip_special_tokens=True)))
        c.append(tokenizer.decode(beam, skip_special_tokens=True))
    cands.extend(c)
    lab.extend([[labels[i]]]*10)
    eval_arg = [inpu,cands,lab]
    if i%100==0:
        metric = metrics_func(eval_arg)
        print(metric)


print(len(inpu),len(cands),len(lab))
metric = metrics_func(eval_arg)
print(metric)


FileNotFoundError: [Errno 2] No such file or directory: '../../data/10/test.csv'

In [3]:
import torch 
text = "'<copy_1.0> <levsim_0.8> <cratio_0.8> The town is mostly constituted of mud houses and small canals that supply water.'"
    # encode the text into tensor of integers using the appropriate tokenizer
input_ids  = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True).to('cuda:1')
#input_ids = tokenizer(inpu, return_tensors='pt', padding=True).input_ids

print(tokenizer.batch_decode(torch.argmax(model(input_ids = input_ids)[0], axis=-1)))

['<s>TheThe is� is� is town is mostly made of mud houses and small canals that supply water.</s>']


In [4]:
max_l = 512
num_b = 10
num_sub_b =1
text = "<copy_1.0> <levsim_0.8> <cratio_0.8> The town is mostly constituted of mud houses and small canals that supply water."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode(text, return_tensors="pt", max_length=512, truncation=True).to('cuda:1')

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(x, tokenizer.decode(beam, skip_special_tokens=True, clean_up_tokenization_spaces=True)))

/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/home2/aparna/miniconda3/envs/new/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:396: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `4` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


0 The town is mostly made up of mud houses and small canals that supply water.
1 The town is mostly made of mud houses and small canals that supply water.
2 The town is made up of mud houses and small canals that supply water.
3 It is mostly made up of mud houses and small canals that supply water.
4 It is made up of mud houses and small canals that supply water.
5 The town is made of mud houses and small canals that supply water.
6 It is mostly made of mud houses and small canals that supply water.
7 The town is mainly made up of mud houses and small canals that supply water.
8 The town is mostly made up of mud houses and small canals.
9 The town is mostly made up of mud houses and small canals that provide water.


In [5]:
#get output for diffrent combination of copy and lev and cratio
for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
            text = "<copy_0.{}> <levsim_0.{}> <cratio_0.{}> The town is mostly constituted of mud houses and small canals that supply water.".format(i,j,k)
            # encode the text into tensor of integers using the appropriate tokenizer
            inputs = tokenizer.encode( text, return_tensors="pt", max_length=512, truncation=True).to('cuda:1')
                
                    # generate text until the output length (which includes the context length) reaches 50
            beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
                no_repeat_ngram_size=3,
                num_return_sequences=10,
                top_k=4, top_p=0.95
                # return_dict_in_generate=True,
            )
       
            print("{} {}".format(f"<copy_0.{i}> <levsim_0.{j}> <cratio_0.{k}>", tokenizer.decode(beam_outputs[0], skip_special_tokens=True)))
                
            
        

<copy_0.1> <levsim_0.1> <cratio_0.1> The town is mostly made up of mud houses and canals.
<copy_0.1> <levsim_0.1> <cratio_0.2> It has mud houses and canals.
<copy_0.1> <levsim_0.1> <cratio_0.3> The town is mostly mud houses.
<copy_0.1> <levsim_0.1> <cratio_0.4> The town is mostly made of mud houses and canals.
<copy_0.1> <levsim_0.1> <cratio_0.5> The town is mostly made up of mud houses and canals.
<copy_0.1> <levsim_0.1> <cratio_0.6> The town is mostly made of mud houses.
<copy_0.1> <levsim_0.1> <cratio_0.7> The town is mostly made of mud houses and small canals.
<copy_0.1> <levsim_0.1> <cratio_0.8> The town is mostly made up of mud houses and small canals.
<copy_0.1> <levsim_0.1> <cratio_0.9> The town is mostly made up of mud houses and small canals that supply water.
<copy_0.1> <levsim_0.2> <cratio_0.1> It has mud houses and canals.
<copy_0.1> <levsim_0.2> <cratio_0.2> It is mostly mud houses.
<copy_0.1> <levsim_0.2> <cratio_0.3> It is mostly mud houses.
<copy_0.1> <levsim_0.2> <cra

KeyboardInterrupt: 

In [14]:
text = "<copy_0.5> <levsim_0.8> <cratio_0.8> Experts say China's air pollution exacts a tremendous toll on human health."
    # encode the text into tensor of integers using the appropriate tokenizer
inputs = tokenizer.encode("paraphrase: " + text, return_tensors="pt", max_length=512, truncation=True).to('cuda:1')

    # generate text until the output length (which includes the context length) reaches 50
beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
    no_repeat_ngram_size=3,
    num_return_sequences=10,
    top_k=4, top_p=0.95
    # return_dict_in_generate=True,
)
for x, beam in enumerate(beam_outputs):
    print("{} {}".format(x, tokenizer.decode(beam, skip_special_tokens=True)))

0 China's air pollution is very bad for human health.
1 China's air pollution is very bad.
2 China's air pollution is very harmful to human health.
3 China's air pollution has a lot of health problems.
4 China's air pollution affects human health.
5 China's air pollution causes a lot of problems.
6 China's air pollution causes a lot of health problems.
7 China's air pollution is a huge problem.
8 China's air pollution is very bad for health.
9 China's air pollution is a very bad thing.


In [11]:
#get output for diffrent combination of copy and lev and cratio
for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
            text = "<copy_0.{}> <levsim_0.{}> <cratio_0.{}> Experts say China's air pollution exacts a tremendous toll on human health.".format(i,j,k)
            # encode the text into tensor of integers using the appropriate tokenizer
            inputs = tokenizer.encode( text, return_tensors="pt", max_length=512, truncation=True).to('cuda:1')
                
                    # generate text until the output length (which includes the context length) reaches 50
            beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
                no_repeat_ngram_size=3,
                num_return_sequences=10,
                top_k=4, top_p=0.95
                # return_dict_in_generate=True,
            )
       
            print("{} {}".format(f"<copy_0.{i}> <levsim_0.{j}> <cratio_0.{k}>", tokenizer.decode(beam_outputs[0], skip_special_tokens=True)))
                
            
        

<copy_0.1> <levsim_0.1> <cratio_0.1> China's air pollution is very bad for human health.
<copy_0.1> <levsim_0.1> <cratio_0.2> There is a lot of air pollution in China.
<copy_0.1> <levsim_0.1> <cratio_0.3> It has a huge toll on human health.
<copy_0.1> <levsim_0.1> <cratio_0.4> China's air pollution has a huge toll on human health.
<copy_0.1> <levsim_0.1> <cratio_0.5> China's air pollution has a huge toll on human health.
<copy_0.1> <levsim_0.1> <cratio_0.6> Experts say China's air pollution is very bad.
<copy_0.1> <levsim_0.1> <cratio_0.7> Experts say China's air pollution has a huge toll on human health.
<copy_0.1> <levsim_0.1> <cratio_0.8> China's air pollution has a huge toll on human health.
<copy_0.1> <levsim_0.1> <cratio_0.9> China's air pollution has a huge toll on human health.
<copy_0.1> <levsim_0.2> <cratio_0.1> There is a lot of air pollution in China.
<copy_0.1> <levsim_0.2> <cratio_0.2> There is a lot of air pollution.
<copy_0.1> <levsim_0.2> <cratio_0.3> China's air pollu

KeyboardInterrupt: 

In [10]:
#get output for diffrent combination of copy and lev and cratio
for i in range(1,10):
    for j in range(1,10):
        for k in range(1,10):
            text = "<copy_0.{}> <levsim_0.{}> <cratio_0.{}>Since,2010, project researchers have uncovered documents in portugal that have revealed who owned the ship.".format(i,j,k)
            # encode the text into tensor of integers using the appropriate tokenizer
            inputs = tokenizer.encode( text, return_tensors="pt", max_length=512, truncation=True).to('cuda:1')
                
                    # generate text until the output length (which includes the context length) reaches 50
            beam_outputs = model.generate(inputs,max_length=max_l,num_beams=num_b,early_stopping=True,
                no_repeat_ngram_size=3,
                num_return_sequences=10,
                top_k=4, top_p=0.95
                # return_dict_in_generate=True,
            )
       
            print("{} {}".format(f"<copy_0.{i}> <levsim_0.{j}> <cratio_0.{k}>", tokenizer.decode(beam_outputs[0], skip_special_tokens=True)))
                
            
        

<copy_0.1> <levsim_0.1> <cratio_0.1> Since 2010, project researchers have uncovered who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.2> The ship was built in 2010.
<copy_0.1> <levsim_0.1> <cratio_0.3> Since 2010, project researchers have discovered who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.4> Since 2010, project researchers have uncovered who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.5> Since 2010, project researchers have uncovered who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.6> Since 2010, project researchers have uncovered who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.7> Since 2010, project researchers have uncovered who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.8> Since 2010, project researchers have found documents in portugal that show who owned the ship.
<copy_0.1> <levsim_0.1> <cratio_0.9> Since 2010, project researchers have found documents in portugal that show who owned the ship.
<copy_0.1> <levsim_0.2> <cratio_0.1> The ship 

## controled